<a href="https://colab.research.google.com/github/JhonatanGttg/Algoritmos-e-Complexidade/blob/main/Extracao_Noticias_JC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio: Extração de Notícias com BeautifulSoup + Requests

**Aluno:** Jhonatan Inacio

In [17]:
# Instalação das bibliotecas necessárias no Google Colab
!pip install requests beautifulsoup4 pandas lxml

In [18]:
# Importação das bibliotecas necessárias para web scraping
import requests  # Para fazer requisições HTTP ao site
from bs4 import BeautifulSoup  # Para analisar e extrair dados do HTML
import pandas as pd  # Para manipulação de dados e criação do CSV
import re  # Para expressões regulares (busca por padrões)
from datetime import datetime  # Para trabalhar com datas
import time  # Para adicionar delays se necessário

print("✅ Todas as bibliotecas foram importadas com sucesso!")

✅ Todas as bibliotecas foram importadas com sucesso!


In [19]:
# Configuração de headers HTTP para simular um navegador real
# Isso ajuda a evitar que o site bloqueie nossa requisição
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8',
    'Connection': 'keep-alive'
}

# URL do site JC UOL conforme especificado na atividade
url = "https://jc.uol.com.br/"

print(f"🌐 Configuração definida para acessar: {url}")
print(f"🤖 User-Agent configurado para simular navegador")

🌐 Configuração definida para acessar: https://jc.uol.com.br/
🤖 User-Agent configurado para simular navegador


In [20]:
def extrair_noticias_jc_uol():
    """
    Função principal para extrair notícias do site JC UOL

    Esta função:
    1. Faz uma requisição HTTP para o site
    2. Analisa o HTML da página
    3. Procura por elementos que contêm notícias
    4. Extrai título, link e data de cada notícia
    5. Retorna uma lista de dicionários

    Returns:
        list: Lista de dicionários com título, link e data das notícias
    """
    try:
        # Passo 1: Fazendo a requisição HTTP para a página inicial do JC UOL
        print("📡 Fazendo requisição para o site JC UOL...")
        response = requests.get(url, headers=headers, timeout=15)

        # Verificando se a requisição foi bem-sucedida (código 200 = OK)
        if response.status_code == 200:
            print(f"✅ Requisição bem-sucedida! Status HTTP: {response.status_code}")
        else:
            print(f"❌ Erro na requisição. Status HTTP: {response.status_code}")
            return []

        # Passo 2: Criando objeto BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        print("🔍 Analisando estrutura HTML da página...")

        # Lista para armazenar todas as notícias extraídas
        noticias = []

        # Passo 3: Estratégia 1 - Buscar por tags <h2> (títulos principais)
        print("🔎 Estratégia 1: Procurando manchetes em tags <h2>...")
        h2_elements = soup.find_all('h2')

        for h2 in h2_elements:
            # Procurando link dentro do h2 ou no elemento pai
            link_element = h2.find('a') or h2.find_parent('a')

            if link_element and h2.get_text(strip=True):
                titulo = h2.get_text(strip=True)  # Extraindo texto limpo
                link = link_element.get('href', '')  # Extraindo URL

                # Convertendo links relativos em URLs completas
                if link.startswith('/'):
                    link = 'https://jc.uol.com.br' + link
                elif not link.startswith('http'):
                    link = 'https://jc.uol.com.br/' + link

                # Usando data atual (pode ser melhorado para extrair data real)
                data = datetime.now().strftime("%d/%m/%Y")

                # Filtro: só adiciona se título tem tamanho razoável
                if titulo and len(titulo) > 10:
                    noticias.append({
                        'titulo': titulo,
                        'link': link,
                        'data': data
                    })

        # Passo 4: Estratégia 2 - Buscar por tags <h3> como backup
        print("🔎 Estratégia 2: Procurando manchetes em tags <h3>...")
        h3_elements = soup.find_all('h3')

        for h3 in h3_elements:
            link_element = h3.find('a') or h3.find_parent('a')

            if link_element and h3.get_text(strip=True):
                titulo = h3.get_text(strip=True)
                link = link_element.get('href', '')

                # Convertendo links relativos
                if link.startswith('/'):
                    link = 'https://jc.uol.com.br' + link
                elif not link.startswith('http'):
                    link = 'https://jc.uol.com.br/' + link

                data = datetime.now().strftime("%d/%m/%Y")

                # Evitando duplicatas: só adiciona se título ainda não existe
                if (titulo and len(titulo) > 10 and
                    not any(n['titulo'] == titulo for n in noticias)):
                    noticias.append({
                        'titulo': titulo,
                        'link': link,
                        'data': data
                    })

        # Passo 5: Estratégia 3 - Buscar links diretos com texto significativo
        print("🔎 Estratégia 3: Procurando links diretos com títulos...")
        links = soup.find_all('a', href=True)

        for link_elem in links:
            titulo = link_elem.get_text(strip=True)
            link = link_elem.get('href', '')

            # Filtros para identificar links de notícias válidos
            if (titulo and len(titulo) > 15 and  # Título com tamanho mínimo
                ('jc.uol.com.br' in link or link.startswith('/')) and  # Link válido
                not any(palavra in titulo.lower() for palavra in ['publicidade', 'anúncio', 'menu'])):

                # Convertendo links relativos
                if link.startswith('/'):
                    link = 'https://jc.uol.com.br' + link
                elif not link.startswith('http'):
                    link = 'https://jc.uol.com.br/' + link

                data = datetime.now().strftime("%d/%m/%Y")

                # Evitando duplicatas
                if not any(n['titulo'] == titulo for n in noticias):
                    noticias.append({
                        'titulo': titulo,
                        'link': link,
                        'data': data
                    })

        # Limitando a 20 notícias para não sobrecarregar
        noticias = noticias[:20]

        print(f"📰 Total de notícias extraídas: {len(noticias)}")
        return noticias

    except requests.RequestException as e:
        print(f"❌ Erro na requisição HTTP: {e}")
        return []
    except Exception as e:
        print(f"❌ Erro inesperado durante extração: {e}")
        return []

In [21]:
def exibir_resultados(noticias):
    """
    Exibe os resultados da extração de forma organizada no terminal

    Esta função formata e apresenta cada notícia extraída
    mostrando título, link e data de forma legível

    Args:
        noticias (list): Lista de dicionários com as notícias extraídas
    """
    print("\n" + "="*80)
    print("📊 RESULTADOS DA EXTRAÇÃO DE NOTÍCIAS DO JC UOL")
    print("="*80)

    # Verificando se há notícias para exibir
    if not noticias:
        print("❌ Nenhuma notícia foi extraída. Verifique a conexão ou estrutura do site.")
        return

    # Exibindo cada notícia numerada
    for i, noticia in enumerate(noticias, 1):
        print(f"\n📰 Notícia {i}:")
        print(f"   📝 Título: {noticia['titulo']}")
        print(f"   🔗 Link: {noticia['link']}")
        print(f"   📅 Data: {noticia['data']}")
        print("-" * 80)

    print(f"\n✅ Total de notícias processadas: {len(noticias)}")

In [22]:
def salvar_csv(noticias, nome_arquivo="noticias_jc_uol.csv"):
    """
    Salva as notícias extraídas em um arquivo CSV

    Esta função converte a lista de dicionários em um DataFrame
    do pandas e salva no formato CSV com encoding UTF-8

    Args:
        noticias (list): Lista de dicionários com as notícias
        nome_arquivo (str): Nome do arquivo CSV a ser criado
    """
    try:
        # Verificando se há notícias para salvar
        if not noticias:
            print("❌ Não há notícias para salvar no CSV.")
            return

        # Criando DataFrame do pandas a partir da lista de dicionários
        df = pd.DataFrame(noticias)

        # Salvando em CSV com encoding UTF-8 (suporte a acentos)
        df.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')

        print(f"💾 Arquivo CSV salvo com sucesso: {nome_arquivo}")
        print(f"📊 Total de registros salvos: {len(noticias)}")

        # Mostrando preview dos primeiros registros
        print("\n📋 Preview dos dados salvos:")
        print(df.head().to_string(index=False, max_colwidth=50))

        # Informações sobre o arquivo
        print(f"\n📁 Estrutura do CSV:")
        print(f"   - Colunas: {list(df.columns)}")
        print(f"   - Linhas: {len(df)}")

    except Exception as e:
        print(f"❌ Erro ao salvar arquivo CSV: {e}")

In [23]:
# 🚀 EXECUÇÃO PRINCIPAL DO PROGRAMA
# Este é o bloco principal que executa todo o processo de extração

def main():
    """
    Função principal que coordena todo o processo de extração de notícias

    Etapas:
    1. Extrai notícias do site JC UOL
    2. Exibe resultados organizados
    3. Salva dados em arquivo CSV
    """
    print("🚀 INICIANDO PROCESSO DE EXTRAÇÃO DE NOTÍCIAS DO JC UOL")
    print("⏱️  Este processo pode levar alguns segundos...")
    print("\n" + "="*60)

    # Etapa 1: Extraindo as notícias do site
    print("\n🔍 ETAPA 1: Extraindo notícias...")
    noticias = extrair_noticias_jc_uol()

    # Etapa 2: Exibindo resultados de forma organizada
    print("\n📊 ETAPA 2: Exibindo resultados...")
    exibir_resultados(noticias)

    # Etapa 3: Salvando dados em CSV
    print("\n💾 ETAPA 3: Salvando em arquivo CSV...")
    salvar_csv(noticias)

    # Relatório final
    print("\n" + "="*60)
    print("✅ PROCESSO CONCLUÍDO COM SUCESSO!")
    print("📁 Verifique o arquivo 'noticias_jc_uol.csv' na pasta de downloads")
    print("🎯 Atividade realizada conforme especificações da imagem")

    # Retornando dados para possível análise posterior
    return noticias

# Executando o programa principal
noticias_extraidas = main()

🚀 INICIANDO PROCESSO DE EXTRAÇÃO DE NOTÍCIAS DO JC UOL
⏱️  Este processo pode levar alguns segundos...


🔍 ETAPA 1: Extraindo notícias...
📡 Fazendo requisição para o site JC UOL...
✅ Requisição bem-sucedida! Status HTTP: 200
🔍 Analisando estrutura HTML da página...
🔎 Estratégia 1: Procurando manchetes em tags <h2>...
🔎 Estratégia 2: Procurando manchetes em tags <h3>...
🔎 Estratégia 3: Procurando links diretos com títulos...
📰 Total de notícias extraídas: 20

📊 ETAPA 2: Exibindo resultados...

📊 RESULTADOS DA EXTRAÇÃO DE NOTÍCIAS DO JC UOL

📰 Notícia 1:
   📝 Título: Após campanha acirrada com 'Manas', Academia Brasileira de Cinema fez anúncio: 'É um filme importantíssimo, que tem que ser dado o devido valor'
   🔗 Link: https://jc.uol.com.br/cultura/2025/09/15/o-agente-secreto-e-escolhido-para-disputar-categoria-de-melhor-filme-internacional-no-oscar.html
   📅 Data: 15/09/2025
--------------------------------------------------------------------------------

📰 Notícia 2:
   📝 Título: Crít

In [24]:
# 🔍 VERIFICAÇÃO FINAL DOS RESULTADOS
# Este bloco permite verificar se tudo foi extraído corretamente

print("🔍 VERIFICAÇÃO FINAL DOS DADOS EXTRAÍDOS")
print("="*50)

if noticias_extraidas:
    print(f"✅ Total de notícias coletadas: {len(noticias_extraidas)}")
    print("\n📋 Estrutura dos dados (conforme especificado na atividade):")

    # Mostrando exemplo da estrutura de dados
    exemplo = noticias_extraidas[0] if noticias_extraidas else {}
    print(f"   - Formato: {type(noticias_extraidas).__name__} (lista de dicionários)")
    print(f"   - Campos por notícia: {list(exemplo.keys()) if exemplo else 'N/A'}")

    # Exemplo de como os dados estão estruturados
    if noticias_extraidas:
        print("\n📝 Exemplo de uma notícia extraída:")
        print(f"   {noticias_extraidas[0]}")
    else:
    print("❌ Nenhuma notícia foi extraída")
    print("💡 Possíveis causas:")
    print("   - Problema de conexão com a internet")
    print("   - Site pode ter mudado sua estrutura")
    print("   - Bloqueio por parte do servidor")

🔍 VERIFICAÇÃO FINAL DOS DADOS EXTRAÍDOS
✅ Total de notícias coletadas: 20

📋 Estrutura dos dados (conforme especificado na atividade):
   - Formato: list (lista de dicionários)
   - Campos por notícia: ['titulo', 'link', 'data']

📝 Exemplo de uma notícia extraída:
   {'titulo': "Após campanha acirrada com 'Manas', Academia Brasileira de Cinema fez anúncio: 'É um filme importantíssimo, que tem que ser dado o devido valor'", 'link': 'https://jc.uol.com.br/cultura/2025/09/15/o-agente-secreto-e-escolhido-para-disputar-categoria-de-melhor-filme-internacional-no-oscar.html', 'data': '15/09/2025'}
